In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sumarização de textos em documentos grandes com LangChain 🦜🔗

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## Visão geral

O resumo de texto é o processo de criar uma versão mais curta de um documento de texto, preservando as informações mais importantes. Isso pode ser útil para várias finalidades, como examinar rapidamente um documento longo, obter a essência de um artigo ou compartilhar um resumo com outras pessoas.

Embora resumir um parágrafo curto seja uma tarefa trivial, há alguns desafios a serem superados se você quiser resumir um documento grande, como um arquivo PDF com várias páginas. 

Neste notebook, você utilizará o LangChain, o framework open source para desenvolver aplicações com LLMs, para aplicar diferentes estratégias de sumarização.

### Objetivo

Neste tutorial, você aprenderá como usar o LangChain em conjunto com as APIs do PaLM2 disponíveis na Vertex AI para resumir informações de texto trabalhando com os seguintes exemplos:

- Método de *stuffing*
- Método MapReduce
- MapReduce com Refinamento

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Generative AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI e outras dependências

In [ ]:
!sudo apt -y -qq install tesseract-ocr
!sudo apt -y -qq install libtesseract-dev
!sudo apt-get -y -qq install poppler-utils #required by PyPDF2 for page count and other pdf utilities
!sudo apt-get -y -qq install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

In [ ]:
import os

! pip3 install --user --upgrade pytesseract pypdf PyPDF2 textract langchain transformers google-cloud-aiplatform

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# PROJECT_ID = "[your-project-name]" # @param {type:"string"}
# REGION = "us-central1"

# import vertexai

# vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

import urllib
import warnings
from pathlib import Path as p

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.llms import VertexAI

warnings.filterwarnings("ignore")

#### Carregando o modelo `text-bison`

In [ ]:
vertex_llm_text = VertexAI(model_name="text-bison@001")

## Summarization with Large Documents

### Preparando arquivos de dados

Para começar, você precisará baixar um arquivo pdf para as tarefas de resumo abaixo.

In [ ]:
# le o arquivo pdf e cria uma lista de páginas
pdf_file = './documentos/documento.pdf'
pdf_loader = PyPDFLoader(pdf_file)
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

## Método 1: Stuffing

A maneira mais simples de passar dados para um modelo de linguagem é enviá-los no prompt como contexto. Isso significa simplesmente incluir todas as informações relevantes no prompt, na ordem em que você deseja que o modelo as processe.

Com o framework LangChain, você usará o método `StuffDocumentsChain` como parte da chamada ao método `load_summarize_chain`. Você necessita somente definir `stuff` como `chain_type` do seu chain.

### Prompt design com um `Stuffing` chain

In [ ]:
prompt_template = """Escreva um sumário conciso do texto abaixo delimitado por três aspas invertidas.
                  Retorne sua resposta em bullets que cubram os pontos principais do texto.
                  ```{text}```
                  SUMARIO EM BULLETS:
                  """

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

Inicie a chain com `stuff` e processe três páginas do documento

In [ ]:
stuff_chain = load_summarize_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)

In [ ]:
three_pages = pages[:3]

In [ ]:
three_pages

In [ ]:
stuff_chain.run(three_pages)

Como pode ser notado, com o método `stuff` você sumariza em uma única chamada todas as informações presentes no seu prompt.

Porém, dependendo do tamanho do seu documento, o método `stuff` pode não funcionar por exceder o limite de input do seu modelo LLM.

In [ ]:
stuff_chain.run(pages)

Como esperado, o código **retorna um erro** devido ao tamanho do contexto do prompt.

### Considerações

O método `stuffing` é uma forma de resumir o texto alimentando o documento inteiro em um modelo de linguagem grande (LLM) em uma única chamada. Este método tem prós e contras.

O método stuffing requer apenas uma única chamada para o LLM, que pode ser mais rápido do que outros métodos que requerem múltiplas chamadas. Ao resumir o texto, o LLM tem acesso a todos os dados de uma só vez, o que pode resultar em um resumo melhor.

Porém, os LLMs têm um limite de contexto, que é o número máximo de tokens que podem ser processados em uma única chamada. Se o documento for maior que o comprimento do contexto, o método de preenchimento não funcionará. Além disso, o método de preenchimento não é adequado para resumir documentos grandes, pois pode ser lento e não produzir um bom resumo.

Vamos explorar outras abordagens para ajudar a lidar com o texto mais longo do que o limite de comprimento de contexto dos LLMs.

## Método 2: MapReduce

O método `MapReduce` implementa um resumo em vários estágios. É uma técnica para resumir grandes trechos de texto, primeiro resumindo trechos menores de texto e depois combinando esses resumos em um único resumo.

Com o framework LangChain, você pode usar o método `MapReduceDocumentsChain` como parte da chamada ao método `load_summarize_chain`. Você só precisa definir `map_reduce` como `chain_type` do seu chain.

### Prompt design com um `MapReduce` chain

Em nosso exemplo, você tem um documento de 25 páginas que precisa resumir.

Com LangChain, na chain `map_reduce` o documento será dividido em no máximo 1.024 pedaços de token. Em seguida, ele executa o prompt inicial definido em cada bloco para gerar um resumo desse bloco. No exemplo abaixo, você usa o primeiro estágio ou prompt do mapa a seguir.

```Escreva um sumário conciso do texto abaixo delimitado por três aspas invertidas. Retorne sua resposta em bullets que cubram os pontos principais do texto. '''{text}''' SUMARIO EM BULLETS:```

Depois que os resumos de todos os blocos são gerados, ele executa um prompt diferente para combinar esses resumos em um único resumo. No exemplo abaixo, você usa o seguinte segundo estágio ou prompt combinado.

```Escreva um sumário conciso do texto abaixo delimitdo por aspas invertidas. Retorne sua resposta em bullets que cubram os pontos principais do texto.```

In [ ]:
map_prompt_template = """
                      Escreva um resumo deste pedaço de texto que inclua os pontos principais e quaisquer detalhes importantes.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Escreva um sumário conciso do texto abaixo delimitdo por aspas invertidas. 
                      Retorne sua resposta em bullets que cubram os pontos principais do texto.
                      ```{text}```
                      SUMARIO EM BULLETS:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

### Gere sumarizações utilizando o método MapReduce

Depois de definir os prompts, você iniciarizará o chain `map_reduce_chain`.

In [ ]:
map_reduce_chain = load_summarize_chain(
    vertex_llm_text,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

Em seguida, você gera resumos usando o chain. Observe que o LangChain utilizará um tokenizer com limite de 1.024 tokens por padrão.

In [ ]:
map_reduce_outputs = map_reduce_chain({"input_documents": pages})

Após a geração dos sumários, você pode validá-los organizando as entradas saídas em um Dataframe do Pandas.

In [ ]:
final_mp_data = []
for doc, out in zip(
    map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_mp_data.append(output)

In [ ]:
pdf_mp_summary = pd.DataFrame.from_dict(final_mp_data)
pdf_mp_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

In [ ]:
index = 3
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])
print("\n\n [Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])

### Considerações

Com o método `MapReduce`, o modelo é capaz de resumir um grande artigo superando o limite de contexto do método `Stuffing` com processamento paralelo.

No entanto, o `MapReduce` requer múltiplas chamadas ao modelo e potencialmente perda de contexto entre as páginas.

Para lidar com esse desafio, você pode tentar outro método para resumir várias páginas por vez.

## Método 3: MapReduce com Refinamento

O método de MapReduce com Refinamento é umaa alternativa para lidar com resumos de documentos grandes. Ele funciona primeiro executando um prompt inicial em um pequeno pedaço de dados, gerando uma saída inicial. Então, para cada documento subsequente, a saída do documento anterior é transmitida junto com o novo documento, e o LLM é solicitado a refinar a saída com base no novo documento.

No LangChain, você pode usar `MapReduceDocumentsChain` como parte da chaamada ao método `load_summarize_chain`. O que você precisa fazer é definir `refine` como `chain_type` da sua cadeia.

### Design de prompt com um `Refine` chain

Usando LangChain, o chain `refine` requer dois prompts.

O prompt de pergunta para gerar a saída para a tarefa subsequente. O prompt de refinamento para refinar a saída com base no conteúdo gerado.

Neste exemplo, o prompt da pergunta é:

```
Forneça um resumo do texto a seguir.
TEXTO: {texto}
RESUMO:
```

e o prompt de refinamento é:

```
Escreva um resumo conciso do texto a seguir delimitado por aspas triplas.
Retorne sua resposta em marcadores que cubram os pontos-chave do texto.
'''{texto}'''
SUMARIO EM BULLETS:
```

In [ ]:
question_prompt_template = """
                  Forneça um resumo do texto a seguir.
                  TEXTO: {text}
                  RESUMO:
                  """

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              Escreva um resumo conciso do texto a seguir delimitado por aspas triplas.
              Retorne sua resposta em marcadores que cubram os pontos-chave do texto.
              ```{text}```
              SUMARIO EM BULLETS:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"]
)

### Gere sumarizações com o método de MapReduce com Refinamento

Depois de definir os prompts, você inicia uma cadeia de resumo usando o tipo de chain `refine`.

In [ ]:
refine_chain = load_summarize_chain(
    vertex_llm_text,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
)

Em seguida, você usará a chain para sumarizar o documento utilizando o método de MapReduce com Refinamento


In [ ]:
refine_outputs = refine_chain({"input_documents": pages})

Below you can see the resulting summaries.

In [ ]:
final_refine_data = []
for doc, out in zip(
    refine_outputs["input_documents"], refine_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_refine_data.append(output)

In [ ]:
pdf_refine_summary = pd.DataFrame.from_dict(final_refine_data)
pdf_refine_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the datafram by filename and page_number
pdf_refine_summary.reset_index(inplace=True, drop=True)
pdf_refine_summary.head()

In [ ]:
index = 3
print("[Context]")
print(pdf_refine_summary["chunks"].iloc[index])
print("\n\n [Simple Summary]")
print(pdf_refine_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_refine_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_refine_summary["file_name"].iloc[index])

### Considerações

O método de MapReduce com Refinamento pode extrair contextos mais relevantes e ter menos perdas do que MapReduce tradicional. No entanto, requer muito mais chamadas para o LLM do que para o Stuffing, e essas chamadas não são independentes, o que significa que não podem ser paralelizadas. Além disso, existe alguma dependência potencial na ordenação dos documentos. Documentos mais recentes podem se tornar mais relevantes, pois este método sofre de viés de atualidade.